In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.co

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

**Recreate the vine_table**

In [ ]:
df = df.dropna()
df = df.dropDuplicates()

vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100WFYHG5LBQZ|          5|            0|          0|   N|                Y|
|R10174PSJAET60|          4|           14|         16|   N|                Y|
|R101DUK4KGSZKO|          5|            1|          1|   N|                N|
|R102BWOXH8BAGB|          5|            0|          0|   N|                Y|
|R102FRHV3HVRYH|          1|            2|          2|   N|                Y|
|R102FVV96K2XPX|          5|            0|          1|   N|                Y|
|R102NGGB4MYAUU|          5|            1|          1|   N|                Y|
|R103DLW6KGTI1T|          5|            4|          4|   N|                Y|
|R103KHSB6CGXI6|          5|            0|          0|   N|                N|
|R103STKNZXONLT|          5|            2|          2|   N|     

In [ ]:
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
# to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

total_votes_df = vine_df.filter("total_votes>=20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JC4MEOAZUXKG|          1|           27|         37|   N|                Y|
| RW6VHD6C7V05E|          5|           30|         31|   N|                N|
|R3EQAM3BUUO3R7|          2|           39|         47|   N|                Y|
|R24M17EDWWOGQY|          2|          105|        124|   N|                Y|
|R3FKGM8DT5EGO5|          4|          230|        232|   N|                Y|
|R3A2ZUDBT9P0EL|          5|           30|         30|   N|                Y|
|R1L2FO5G71HEI0|          1|           19|         21|   N|                N|
|R17JB3EEJ6NNT1|          2|           74|         77|   N|                Y|
|R1OJ6PLL53VH8K|          5|           45|         50|   N|                N|
| R9RAU5XCLE64R|          4|           11|         27|   N|     

In [ ]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of 
# helpful_votes divided by total_votes is equal to or greater than 50%.
from pyspark.sql.functions import round, col
helpful_ratio_df = total_votes_df.withColumn('helpful_ratio', round(total_votes_df['helpful_votes']/ total_votes_df['total_votes']*100, 2))
helpful_df = helpful_ratio_df.filter("helpful_ratio>= 50")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R3JC4MEOAZUXKG|          1|           27|         37|   N|                Y|        72.97|
| RW6VHD6C7V05E|          5|           30|         31|   N|                N|        96.77|
|R3EQAM3BUUO3R7|          2|           39|         47|   N|                Y|        82.98|
|R24M17EDWWOGQY|          2|          105|        124|   N|                Y|        84.68|
|R3FKGM8DT5EGO5|          4|          230|        232|   N|                Y|        99.14|
|R3A2ZUDBT9P0EL|          5|           30|         30|   N|                Y|        100.0|
|R1L2FO5G71HEI0|          1|           19|         21|   N|                N|        90.48|
|R17JB3EEJ6NNT1|          2|           74|         77|   N|                Y|   

In [ ]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was 
# written as part of the Vine program (paid), vine == 'Y'.
vine_reviewed_df = helpful_df.filter("vine = 'Y'")
vine_reviewed_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R30IWXU9RENY1Y|          3|           15|         24|   Y|                N|         62.5|
|R39IHYTIR8S03C|          4|           29|         36|   Y|                N|        80.56|
|R2G965AP9Q5P4J|          5|           21|         23|   Y|                N|         91.3|
|R3T621TI8ZPHKS|          1|          123|        152|   Y|                N|        80.92|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|        100.0|
|R2EOSZN3YYDV6L|          4|           27|         31|   Y|                N|         87.1|
|R354Z6LHDF3XMO|          5|           37|         40|   Y|                N|         92.5|
|R1S9CD7RIIIBR3|          5|           30|         33|   Y|                N|   

In [ ]:
# 4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
vine_not_reviewed_df = helpful_df.filter("vine = 'N'")
vine_not_reviewed_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R3JC4MEOAZUXKG|          1|           27|         37|   N|                Y|        72.97|
| RW6VHD6C7V05E|          5|           30|         31|   N|                N|        96.77|
|R3EQAM3BUUO3R7|          2|           39|         47|   N|                Y|        82.98|
|R24M17EDWWOGQY|          2|          105|        124|   N|                Y|        84.68|
|R3FKGM8DT5EGO5|          4|          230|        232|   N|                Y|        99.14|
|R3A2ZUDBT9P0EL|          5|           30|         30|   N|                Y|        100.0|
|R1L2FO5G71HEI0|          1|           19|         21|   N|                N|        90.48|
|R17JB3EEJ6NNT1|          2|           74|         77|   N|                Y|   

**5. Utilizing Filtered Reviews from Above**
See following cell for Number 5 completed for ALL reviews

In [ ]:
#5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

#Count of all reviews
total_review_count1 = helpful_df.count()

print(f'The total number of reviews is {total_review_count1}')

#Count of 5-star reviews
five_star_df1 = helpful_df.filter("star_rating == 5")
five_star_count1 = five_star_df1.count()
print(f'The total number of five star reviews is {five_star_count1}')


# Paid five star percent
all_paid_df1 = helpful_df.filter("vine = 'Y'")
paid_count1 = all_paid_df1.count()
paid_star_group_df1= all_paid_df1.groupby("star_rating").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "paid_review_id_count")
paid_percents_df1 = paid_star_group_df1.withColumn("star_percentages", round(paid_star_group_df1['paid_review_id_count']/paid_count1*100, 2))

print(f"The percentages of star reviews for all paid reviews is:")
paid_percents_df1.orderBy(paid_star_group_df1["star_rating"].desc()).show()

# Unpaid five star percent
all_unpaid_df1 = helpful_df.filter("vine = 'N'")
unpaid_count1 = all_unpaid_df1.count()
unpaid_star_group_df1 = all_unpaid_df1.groupby("star_rating").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "unpaid_review_id_count")
unpaid_percents_df1 = unpaid_star_group_df1.withColumn("star_percentages", round(unpaid_star_group_df1['unpaid_review_id_count']/unpaid_count1*100, 2))

print(f"The percentages of star reviews for all unpaid reviews is:")
unpaid_percents_df1.orderBy(unpaid_star_group_df1["star_rating"].desc()).show()


# Total Number of Vine Reviews
total_vine_grouped_df1 = helpful_df.groupby("vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "paid_unpaid_review_id_count")
total_vine_grouped_df1.show()


#Percentage of 5-star reviews versus all reviews - paid vs unpaid
vine_grouped_df1 = five_star_df1.groupby("vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "five_star_review_id_count")
vine_five_star_percents1 = vine_grouped_df1.withColumn("five_star_percentages", round(vine_grouped_df1['five_star_review_id_count']/total_review_count1*100, 2))
vine_five_star_percents1.show()

#Percentage of all star reviews - paid vs unpaid
star_group_df1 = helpful_df.groupby("star_rating", "vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "review_id_count")
star_group_percents = star_group_df1.withColumn("star_percentages", round(star_group_df1['review_id_count']/total_review_count1*100, 2))
star_group_percents.orderBy(star_group_df1["star_rating"].desc()).show()



The total number of reviews is 28339
The total number of five star reviews is 15706
The percentages of star reviews for all paid reviews is:
+-----------+--------------------+----------------+
|star_rating|paid_review_id_count|star_percentages|
+-----------+--------------------+----------------+
|          5|                  20|           32.79|
|          4|                  17|           27.87|
|          3|                  11|           18.03|
|          2|                   6|            9.84|
|          1|                   7|           11.48|
+-----------+--------------------+----------------+

The percentages of star reviews for all unpaid reviews is:
+-----------+----------------------+----------------+
|star_rating|unpaid_review_id_count|star_percentages|
+-----------+----------------------+----------------+
|          5|                 15686|           55.47|
|          4|                  2599|            9.19|
|          3|                  1931|            6.83|
|      

**5. Utilizing ALL Reviews**

In [ ]:
#5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

#Count of all reviews
total_review_count = vine_df.count()

print(f'The total number of reviews is {total_review_count}')

#Count of 5-star reviews
five_star_df = vine_df.filter("star_rating == 5")
five_star_count = five_star_df.count()
print(f'The total number of five star reviews is {five_star_count}')


# Paid five star percent
all_paid_df = vine_df.filter("vine = 'Y'")
paid_count = all_paid_df.count()
paid_star_group_df= all_paid_df.groupby("star_rating").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "paid_review_id_count")
paid_percents_df = paid_star_group_df.withColumn("star_percentages", round(paid_star_group_df['paid_review_id_count']/paid_count*100, 2))

print(f"The percentages of star reviews for all paid reviews is:")
paid_percents_df.orderBy(paid_star_group_df["star_rating"].desc()).show()

# Unpaid five star percent
all_unpaid_df = vine_df.filter("vine = 'N'")
unpaid_count = all_unpaid_df.count()
unpaid_star_group_df= all_unpaid_df.groupby("star_rating").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "unpaid_review_id_count")
unpaid_percents_df = unpaid_star_group_df.withColumn("star_percentages", round(unpaid_star_group_df['unpaid_review_id_count']/unpaid_count*100, 2))

print(f"The percentages of star reviews for all unpaid reviews is:")
unpaid_percents_df.orderBy(unpaid_star_group_df["star_rating"].desc()).show()


# Total Number of Vine Reviews
total_vine_grouped_df = vine_df.groupby("vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "paid_unpaid_review_id_count")
total_vine_grouped_df.show()


#Percentage of 5-star reviews versus all reviews - paid vs unpaid
vine_grouped_df = five_star_df.groupby("vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "five_star_review_id_count")
vine_five_star_percents = vine_grouped_df.withColumn("five_star_percentages", round(vine_grouped_df['five_star_review_id_count']/total_review_count*100, 2))
vine_five_star_percents.show()

#Percentage of all star reviews - paid vs unpaid
star_group_df = vine_df.groupby("star_rating", "vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "review_id_count")
star_group_percents = star_group_df.withColumn("star_percentages", round(star_group_df['review_id_count']/total_review_count*100, 2))
star_group_percents.orderBy(star_group_df["star_rating"].desc()).show()


The total number of reviews is 2402212
The total number of five star reviews is 1662118
The percentages of star reviews for all paid reviews is:
+-----------+--------------------+----------------+
|star_rating|paid_review_id_count|star_percentages|
+-----------+--------------------+----------------+
|          5|                5657|           34.06|
|          4|                5965|           35.91|
|          3|                3324|           20.01|
|          2|                1223|            7.36|
|          1|                 442|            2.66|
+-----------+--------------------+----------------+

The percentages of star reviews for all unpaid reviews is:
+-----------+----------------------+----------------+
|star_rating|unpaid_review_id_count|star_percentages|
+-----------+----------------------+----------------+
|          5|               1656461|           69.44|
|          4|                287396|           12.05|
|          3|                158155|            6.63|
|  

In [ ]:
#Breakdown of 5-star reviews - paid vs unpaid
five_star_grouped_df = five_star_df.groupby("vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "review_id_count")
five_star_percents = five_star_grouped_df.withColumn("five_star_percentages", round(five_star_grouped_df['review_id_count']/five_star_count*100, 2))
five_star_percents.show()

+----+---------------+---------------------+
|vine|review_id_count|five_star_percentages|
+----+---------------+---------------------+
|   Y|           5657|                 0.34|
|   N|        1656461|                99.66|
+----+---------------+---------------------+

